# Simulation 2a: Fitting the Modality Effect in 12-Item Lists
This notebook runs Simulation 2 from the paper - fitting the modality effect in 12-item lists.

In [ ]:
import os
import re
import json
import itertools
import numpy as np
import pickle as pkl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from glob import glob
from cluster_helper.cluster import cluster_view

In [ ]:
SMALL_SIZE = 12
MEDIUM_SIZE = 14
LARGE_SIZE = 18

plt.rc('font', size=LARGE_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=LARGE_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=LARGE_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=LARGE_SIZE)  # fontsize of the figure title

COLOR = False

if COLOR:
    VIS_FMT = '#011F5B'
    AUD_FMT = '#990000'
    ERR_VIS = '#011F5B'
    ERR_AUD = '#990000'
    ERR_ALPHA = .3
    ERR_ALPHA2 = .2
else:
    VIS_FMT = 'k-'
    AUD_FMT = 'k--'
    ERR_VIS = 'k'
    ERR_AUD = 'k'
    ERR_ALPHA = .12
    ERR_ALPHA2 = .075
    
def plot_spcs(vis, aud, vis_sim, aud_sim):

    plt.figure(figsize=(12, 10))
    
    # SPC Data
    ax=plt.subplot(3, 2, 1)
    plt.title('Data')
    for ll in vis['spc']:
        plt.plot(range(1, int(ll)+1), np.array(vis['spc'][ll]), VIS_FMT)
        plt.fill_between(range(1, int(ll)+1), np.add(vis['spc'][ll], vis['spc_sem'][ll]), np.subtract(vis['spc'][ll], vis['spc_sem'][ll]), alpha=ERR_ALPHA, color=ERR_VIS)
        plt.plot(range(1, int(ll)+1), np.array(aud['spc'][ll]), AUD_FMT)
        plt.fill_between(range(1, int(ll)+1), np.add(aud['spc'][ll], aud['spc_sem'][ll]), np.subtract(aud['spc'][ll], aud['spc_sem'][ll]), alpha=ERR_ALPHA, color=ERR_AUD)
    plt.xticks([1, 3, 6, 9, 12])
    plt.xlabel('Serial Position')
    plt.ylabel('Recall Prob.')
    plt.ylim(0, 1)
    plt.legend(['Visual', 'Auditory'])
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.yaxis.set_ticks_position('left')
    ax.xaxis.set_ticks_position('bottom')
    
    # SPC Model
    ax=plt.subplot(3, 2, 2)
    plt.title('Model')
    for ll in vis_sim['spc']:
        plt.plot(range(1, int(ll)+1), np.array(vis_sim['spc'][ll]), VIS_FMT)
        plt.fill_between(range(1, int(ll)+1), np.add(vis_sim['spc'][ll], vis_sim['spc_sem'][ll]), np.subtract(vis_sim['spc'][ll], vis_sim['spc_sem'][ll]), alpha=ERR_ALPHA, color=ERR_VIS)
        plt.plot(range(1, int(ll)+1), np.array(aud_sim['spc'][ll]), AUD_FMT)
        plt.fill_between(range(1, int(ll)+1), np.add(aud_sim['spc'][ll], aud_sim['spc_sem'][ll]), np.subtract(aud_sim['spc'][ll], aud_sim['spc_sem'][ll]), alpha=ERR_ALPHA, color=ERR_AUD)
    plt.xticks([1, 3, 6, 9, 12])
    plt.xlabel('Serial Position')
    plt.ylabel('Recall Prob.')
    plt.ylim(0, 1)
    plt.legend(['Visual', 'Auditory'])
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.yaxis.set_ticks_position('left')
    ax.xaxis.set_ticks_position('bottom')
    
    # SPC (FR=SP1) Data
    ax=plt.subplot(3, 2, 3)
    for ll in vis['spc_fr1']:
        plt.plot(range(1, int(ll)+1), vis['spc_fr1'][ll], 'k-')
        plt.fill_between(range(1, int(ll)+1), np.add(vis['spc_fr1'][ll], vis['spc_fr1_sem'][ll]), np.subtract(vis['spc_fr1'][ll], vis['spc_fr1_sem'][ll]), alpha=.12, color='k')
        plt.plot(range(1, int(ll)+1), aud['spc_fr1'][ll], 'k--')
        plt.fill_between(range(1, int(ll)+1), np.add(aud['spc_fr1'][ll], aud['spc_fr1_sem'][ll]), np.subtract(aud['spc_fr1'][ll], aud['spc_fr1_sem'][ll]), alpha=.12, color='k')
    plt.ylim(-.01, 1.01)
    plt.xticks([1, 3, 6, 9, 12])
    plt.xlabel('Serial Position')
    plt.ylabel('Recall Prob. (SP1)')
    plt.ylim(0, 1)
    plt.legend(['Visual', 'Auditory'], loc=3)
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.yaxis.set_ticks_position('left')
    ax.xaxis.set_ticks_position('bottom')
    
    # SPC (FR=SP1) Model
    ax=plt.subplot(3, 2, 4)
    for ll in vis_sim['spc_fr1']:
        plt.plot(range(1, int(ll)+1), vis_sim['spc_fr1'][ll], 'k-')
        plt.fill_between(range(1, int(ll)+1), np.add(vis_sim['spc_fr1'][ll], vis_sim['spc_fr1_sem'][ll]), np.subtract(vis_sim['spc_fr1'][ll], vis_sim['spc_fr1_sem'][ll]), alpha=.12, color='k')
        plt.plot(range(1, int(ll)+1), aud_sim['spc_fr1'][ll], 'k--')
        plt.fill_between(range(1, int(ll)+1), np.add(aud_sim['spc_fr1'][ll], aud_sim['spc_fr1_sem'][ll]), np.subtract(aud_sim['spc_fr1'][ll], aud_sim['spc_fr1_sem'][ll]), alpha=.12, color='k')
    plt.ylim(-.01, 1.01)
    plt.xticks([1, 3, 6, 9, 12])
    plt.xlabel('Serial Position')
    plt.ylabel('Recall Prob. (SP1)')
    plt.ylim(0, 1)
    plt.legend(['Visual', 'Auditory'])#, loc=3)
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.yaxis.set_ticks_position('left')
    ax.xaxis.set_ticks_position('bottom')
    
    # SPC (FR=L4) Data
    ax=plt.subplot(3, 2, 5)
    for ll in vis['spc_frl4']:
        plt.plot(range(1, int(ll)+1), vis['spc_frl4'][ll], 'k-')
        plt.fill_between(range(1, int(ll)+1), np.add(vis['spc_frl4'][ll], vis['spc_frl4_sem'][ll]), np.subtract(vis['spc_frl4'][ll], vis['spc_frl4_sem'][ll]), alpha=.12, color='k')
        plt.plot(range(1, int(ll)+1), aud['spc_frl4'][ll], 'k--')
        plt.fill_between(range(1, int(ll)+1), np.add(aud['spc_frl4'][ll], aud['spc_frl4_sem'][ll]), np.subtract(aud['spc_frl4'][ll], aud['spc_frl4_sem'][ll]), alpha=.12, color='k')
    plt.ylim(-.01, 1.01)
    plt.xticks([1, 3, 6, 9, 12])
    plt.xlabel('Serial Position')
    plt.ylabel('Recall Prob. (L4)')
    plt.ylim(0, 1)
    plt.legend(['Visual', 'Auditory'], loc=0)
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.yaxis.set_ticks_position('left')
    ax.xaxis.set_ticks_position('bottom')
    
    # SPC (FR=L4) Model
    ax=plt.subplot(3, 2, 6)
    for ll in vis_sim['spc_frl4']:
        plt.plot(range(1, int(ll)+1), vis_sim['spc_frl4'][ll], 'k-')
        plt.fill_between(range(1, int(ll)+1), np.add(vis_sim['spc_frl4'][ll], vis_sim['spc_frl4_sem'][ll]), np.subtract(vis_sim['spc_frl4'][ll], vis_sim['spc_frl4_sem'][ll]), alpha=.12, color='k')
        plt.plot(range(1, int(ll)+1), aud_sim['spc_frl4'][ll], 'k--')
        plt.fill_between(range(1, int(ll)+1), np.add(aud_sim['spc_frl4'][ll], aud_sim['spc_frl4_sem'][ll]), np.subtract(aud_sim['spc_frl4'][ll], aud_sim['spc_frl4_sem'][ll]), alpha=.12, color='k')
    plt.ylim(-.01, 1.01)
    plt.xticks([1, 3, 6, 9, 12])
    plt.xlabel('Serial Position')
    plt.ylabel('Recall Prob. (L4)')
    plt.ylim(0, 1)
    plt.legend(['Visual', 'Auditory'], loc=0)
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.yaxis.set_ticks_position('left')
    ax.xaxis.set_ticks_position('bottom')
    
    plt.tight_layout(h_pad=-.05)


def plot_nonspc(vis, aud, vis_sim, aud_sim):
    
    plt.figure(figsize=(12, 10))
    
    # PFR Data
    ax=plt.subplot(321)
    plt.title('Data')
    for ll in vis['pfr']:
        plt.plot(range(1, int(ll)+1), vis['pfr'][ll], VIS_FMT)
        plt.fill_between(range(1, int(ll)+1), np.add(vis['pfr'][ll], vis['pfr_sem'][ll]), np.subtract(vis['pfr'][ll], vis['pfr_sem'][ll]), alpha=ERR_ALPHA, color=ERR_VIS)
        plt.plot(range(1, int(ll)+1), aud['pfr'][ll], AUD_FMT)
        plt.fill_between(range(1, int(ll)+1), np.add(aud['pfr'][ll], aud['pfr_sem'][ll]), np.subtract(aud['pfr'][ll], aud['pfr_sem'][ll]), alpha=ERR_ALPHA, color=ERR_AUD)
    plt.xticks([1, 3, 6, 9, 12])
    plt.xlabel('Serial Position')
    plt.ylabel('Prob. of First Recall')
    plt.legend(['Visual', 'Auditory'])
    plt.ylim(0, .5)
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.yaxis.set_ticks_position('left')
    ax.xaxis.set_ticks_position('bottom')
    
    # PFR Model
    ax=plt.subplot(322)
    plt.title('Model')
    for ll in vis_sim['pfr']:
        plt.plot(range(1, int(ll)+1), vis_sim['pfr'][ll], VIS_FMT)
        plt.fill_between(range(1, int(ll)+1), np.add(vis_sim['pfr'][ll], vis_sim['pfr_sem'][ll]), np.subtract(vis_sim['pfr'][ll], vis_sim['pfr_sem'][ll]), alpha=ERR_ALPHA, color=ERR_VIS)
        plt.plot(range(1, int(ll)+1), aud_sim['pfr'][ll], AUD_FMT)
        plt.fill_between(range(1, int(ll)+1), np.add(aud_sim['pfr'][ll], aud_sim['pfr_sem'][ll]), np.subtract(aud_sim['pfr'][ll], aud_sim['pfr_sem'][ll]), alpha=ERR_ALPHA, color=ERR_AUD)
    plt.xticks([1, 3, 6, 9, 12])
    plt.xlabel('Serial Position')
    plt.ylabel('Prob. of First Recall')
    plt.legend(['Visual', 'Auditory'], loc=2)
    plt.ylim(0, .5)
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.yaxis.set_ticks_position('left')
    ax.xaxis.set_ticks_position('bottom')
    
    # PLIs Data
    ax=plt.subplot(323)
    plt.bar([1, 1.75], [vis['plis'], aud['plis']], yerr=[vis['plis_sem']*1.96, aud['plis_sem']*1.96], color=['#a3a3a3', 'w'], ec='k', width=.5, capsize=3)
    plt.xlim(.5, 2.25)
    plt.xticks([1, 1.75], ['Visual', 'Auditory'])
    plt.xlabel('Modality')
    plt.ylabel('PLIs per Trial')
    plt.ylim(0, .3)
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.yaxis.set_ticks_position('left')
    ax.xaxis.set_ticks_position('bottom')
    
    # PLIs Model
    ax=plt.subplot(324)
    plt.bar([1, 1.75], [vis_sim['plis'], aud_sim['plis']], yerr=[vis_sim['plis_sem']*1.96, aud_sim['plis_sem']*1.96], color=['#a3a3a3', 'w'], ec='k', width=.5, capsize=3)
    plt.xlim(.5, 2.25)
    plt.xticks([1, 1.75], ['Visual', 'Auditory'])
    plt.xlabel('Modality')
    plt.ylabel('PLIs per Trial')
    plt.ylim(0, .3)
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.yaxis.set_ticks_position('left')
    ax.xaxis.set_ticks_position('bottom')
    
    # PLI Recency Data
    ax=plt.subplot(325)
    plt.plot(range(1, 6), vis['pli_recency'], VIS_FMT)
    plt.fill_between(range(1, 6), np.add(vis['pli_recency'], vis['pli_recency_sem']), np.subtract(vis['pli_recency'], vis['pli_recency_sem']), alpha=ERR_ALPHA, color=ERR_VIS)
    plt.plot(range(1, 6), aud['pli_recency'], AUD_FMT)
    plt.fill_between(range(1, 6), np.add(aud['pli_recency'], aud['pli_recency_sem']), np.subtract(aud['pli_recency'], aud['pli_recency_sem']), alpha=ERR_ALPHA, color=ERR_AUD)
    plt.xlabel('List Recency')
    plt.ylabel('Proportion of PLIs')
    plt.legend(['Visual', 'Auditory'])
    plt.ylim(0, .5)
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.yaxis.set_ticks_position('left')
    ax.xaxis.set_ticks_position('bottom')
    
    # PLI Recency Model
    ax=plt.subplot(326)
    plt.plot(range(1, 6), vis_sim['pli_recency'], VIS_FMT)
    plt.fill_between(range(1, 6), np.add(vis_sim['pli_recency'], vis_sim['pli_recency_sem']), np.subtract(vis_sim['pli_recency'], vis_sim['pli_recency_sem']), alpha=ERR_ALPHA, color=ERR_VIS)
    plt.plot(range(1, 6), aud_sim['pli_recency'], AUD_FMT)
    plt.fill_between(range(1, 6), np.add(aud_sim['pli_recency'], aud_sim['pli_recency_sem']), np.subtract(aud_sim['pli_recency'], aud_sim['pli_recency_sem']), alpha=ERR_ALPHA, color=ERR_AUD)
    plt.xlabel('List Recency')
    plt.ylabel('Proportion of PLIs')
    plt.legend(['Visual', 'Auditory'])
    plt.ylim(0, .5)
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.yaxis.set_ticks_position('left')
    ax.xaxis.set_ticks_position('bottom')
    
    plt.tight_layout(h_pad=-.05)


def eval_model(param_vec):
    
    import os
    import sys
    import json
    import numpy as np
    import pickle as pkl
    import CMR2_pack_cyth as CMR2    
    from glob import glob
    
    sys.path.append('../CMR2/fitting/')
    import optimization_utils as opt
    
    def chi_squared_error(target_stats, cmr_stats):
    
        y = []
        y_sem = []
        y_hat = []

        # Fit SPC and PFR
        for stat in ('spc_fr1', 'spc_frl4', 'pfr'):
            for ll in cmr_stats[stat]:
                # Skip serial position 1 for SPC when initiating recall from position 1 (to avoid dividing by 0 standard error, since prec is always 1 by definition)
                if stat == 'spc_fr1':
                    y.append(np.atleast_1d(target_stats[stat][ll][1:]))
                    y_sem.append(np.atleast_1d(target_stats[stat + '_sem'][ll][1:]))
                    y_hat.append(np.atleast_1d(cmr_stats[stat][ll][1:]))
                else:
                    y.append(np.atleast_1d(target_stats[stat][ll]))
                    y_sem.append(np.atleast_1d(target_stats[stat + '_sem'][ll]))
                    y_hat.append(np.atleast_1d(cmr_stats[stat][ll]))

        # Fit PLIs and PLI recency (not separated by list length)
        for stat in ('plis', 'pli_recency'):
            y.append(np.atleast_1d(target_stats[stat]))
            y_sem.append(np.atleast_1d(target_stats[stat + '_sem']))
            y_hat.append(np.atleast_1d(cmr_stats[stat]))

        y = np.concatenate(y)
        y_sem = np.concatenate(y_sem)
        y_hat = np.concatenate(y_hat)

        chi2_err = np.mean(((y - y_hat) / y_sem) ** 2)
        chi2_err = np.mean(((y - y_hat) / y_sem) ** 2)
        spc_fr1_err = np.mean(((target_stats['spc_fr1']['12'][1:] - cmr_stats['spc_fr1']['12'][1:]) / target_stats['spc_fr1_sem']['12'][1:]) ** 2)
        spc_frl4_err = np.mean(((target_stats['spc_frl4']['12'] - cmr_stats['spc_frl4']['12']) / target_stats['spc_frl4_sem']['12']) ** 2)
        pfr_err = np.mean(((target_stats['pfr']['12'] - cmr_stats['pfr']['12']) / target_stats['pfr_sem']['12']) ** 2)
        pli_err = np.mean(((target_stats['plis'] - cmr_stats['plis']) / target_stats['plis_sem'])  ** 2)
        plir_err = np.mean(((target_stats['pli_recency'] - cmr_stats['pli_recency']) / target_stats['pli_recency_sem'])  ** 2)

        return chi2_err, spc_fr1_err, spc_frl4_err, pfr_err, pli_err, plir_err
    
    ##########
    #
    # Initialization
    #
    ##########
    
    # Settings
    n_sess = 1469  # Number of sessions to simulate -- max 1469
    n_runs = 5 # Number of times to simulate each session
    wordpool_file = '../CMR2/wasnorm_wordpool.txt'  # Path to word pool file
    w2v_file = '../CMR2/w2v.txt'  # Path to semantic associative matrix file
    target_stat_file_v = '../CMR2/target_stats_sim2a_v.json'  # Target stats for visual simulation
    target_stat_file_a = '../CMR2/target_stats_sim2a_a.json'  # Target statas for auditory simulation
        
    # Load data
    file_list = glob('/data/eeg/scalp/ltp/ltpFR3_MTurk/data/MTK*.json')
    wn = set(np.loadtxt('/data/eeg/scalp/ltp/ltpFR3_MTurk/WROTE_NOTES.txt', dtype=str))
    file_list = sorted([f for f in file_list if int(f[-9:-5]) > 1308 and f[-12:-5] not in wn])
    file_list = file_list[:n_sess]
    file_list = file_list * n_runs
    data_pres, sessions, sources = opt.get_data(file_list, wordpool_file, fixed_length=True)
    sources = None
    
    # Load semantic similarity matrix (word2vec)
    w2v = np.loadtxt(w2v_file)

    # Load target stats from JSON file
    with open(target_stat_file_v, 'r') as f:
        target_stats_v = json.load(f)
    with open(target_stat_file_a, 'r') as f:
        target_stats_a = json.load(f)
    for key in target_stats_v:
        if isinstance(target_stats_v[key], list):
            target_stats_v[key] = np.array(target_stats_v[key], dtype=float)
            target_stats_a[key] = np.array(target_stats_a[key], dtype=float)
        elif isinstance(target_stats_v[key], dict):
            for subkey in target_stats_v[key]:
                if isinstance(target_stats_v[key][subkey], list):
                    target_stats_v[key][subkey] = np.array(target_stats_v[key][subkey], dtype=float)
                    target_stats_a[key][subkey] = np.array(target_stats_a[key][subkey], dtype=float)
    
    # Extract parallel ID number if one was provided in the parameter vector
    if len(param_vec) == 15:
        parallel_ID = int(param_vec[-1])
        param_vec = param_vec[:-1]
    else:
        parallel_ID = None
    
    ##########
    #
    # Run Model
    #
    ##########
    
    # Run model with the parameters given in param_vec
    data_path = '/scratch/jpazdera/cmr2/param_saves/sim2a/data%s.pkl' % parallel_ID
    if os.path.exists(data_path):
        with open(data_path, 'rb') as f:
            cmr_stats = pkl.load(f)
    else:
        _, cmr_stats = opt.obj_func(param_vec, target_stats_v, data_pres, sessions, w2v, sources, return_recalls=False)
        
        # Score against average visual stats
        scores = chi_squared_error(target_stats_v, cmr_stats)
        cmr_stats['err_v'] = scores[0]
        cmr_stats['err_spc_composite_v'] = 11/23 * scores[1] + 12/23 * scores[2]
        cmr_stats['err_spc_fr1_v'] = scores[1]
        cmr_stats['err_spc_frl4_v'] = scores[2]
        cmr_stats['err_pfr_v'] = scores[3]
        cmr_stats['err_pli_v'] = scores[4]
        cmr_stats['err_plir_v'] = scores[5]
        
        # Score against average auditory stats
        scores = chi_squared_error(target_stats_a, cmr_stats)
        cmr_stats['err_a'] = scores[0]
        cmr_stats['err_spc_composite_a'] = 11/23 * scores[1] + 12/23 * scores[2]
        cmr_stats['err_spc_fr1_a'] = scores[1]
        cmr_stats['err_spc_frl4_a'] = scores[2]
        cmr_stats['err_pfr_a'] = scores[3]
        cmr_stats['err_pli_a'] = scores[4]
        cmr_stats['err_plir_a'] = scores[5]
        
        # Save results
        with open(data_path, 'wb') as f:
            pkl.dump(cmr_stats, f)
    
    return cmr_stats

# Get Target Stats

### Variable List Length

In [ ]:
# Load Experiment 2 average stats
with open('/data/eeg/scalp/ltp/ltpFR3_MTurk/stats/all_v2_excl_wn.json', 'r') as f:
    d = json.load(f)

m = d['mean']
s = d['sem']
for mod in ('v', 'a'):
    targets = {}
    for stat in ('spc', 'spc_fr1', 'spc_frl4', 'pfr'):
        targets[stat] = {}
        targets[stat + '_sem'] = {}
        for list_length in ('12', '24'):
            targets[stat][list_length] = m[stat][mod + list_length]
            targets[stat + '_sem'][list_length] = s[stat][mod + list_length]
        
    # Get the stats that will not be split by list length 
    for stat in ('plis', 'pli_recency'):
        targets[stat] = m[stat][mod]
        targets[stat + '_sem'] = s[stat][mod]

    # Save stats to a JSON file for access by the particle swarm
    with open('../CMR2/target_stats_sim2a_%s.json' % mod, 'w') as f:
        json.dump(targets, f)

### Fixed List Length

In [ ]:
# Load Experiment 2 average stats for 12-item lists
with open('/data/eeg/scalp/ltp/ltpFR3_MTurk/stats/all_v2_excl_wn.json', 'r') as f:
    d = json.load(f)
    
m = d['mean']
s = d['sem']
for mod in ('v', 'a'):
    targets = {}
    for stat in ('spc', 'spc_fr1', 'spc_frl4', 'pfr'):
        targets[stat] = {}
        targets[stat + '_sem'] = {}
        targets[stat]['12'] = m[stat][mod + '12']
        targets[stat + '_sem']['12'] = s[stat][mod + '12']
    
    for stat in ('plis', 'pli_recency'):
        targets[stat] = m[stat][mod + '12']
        targets[stat + '_sem'] = s[stat][mod + '12']

    # Save stats to a JSON file for access by the particle swarm
    with open('../CMR2/target_stats_sim2a_%s.json' % mod, 'w') as f:
        json.dump(targets, f)

# Set Base Parameters

This simulation uses the parameters from Fit 101.

In [ ]:
param_labels = [
    r'$\beta_{enc}$',
    r'$\beta_{rec}$',
    r'$\gamma_{FC}$',
    r'$\gamma_{CF}$',
    r'$\phi_{s}$',
    r'$\phi_{d}$',
    r'$\kappa$',
    r'$\eta$',
    r'$s$',
    r'$\beta_{post}$',
    r'$\omega$',
    r'$\alpha$',
    r'$c_{thresh}$',
    r'$\lambda$',
    r'$\beta_{source}$',
    r'$\gamma_{source}$'
]

# Use the parameters from Fit 101
params = [
    0.43876,
    0.2059,
    0.16544,
    0.53347,
    2.51461,
    0.53605,
    0.06862,
    0.08219,
    1.9117,
    0.69074,
    7.68183,
    0.84223,
    0.02197,
    0.06274
]


# Grid Search (Beta Encoding x Gamma CF)

### Create grid

In [ ]:
tests_per_dim = 101

lb = [params[0] - .1, params[3] - .15]
ub = [params[0] + .1, params[3] + .15]

param_sets = np.array([np.append(params, i) for i in range(tests_per_dim**2)])
p1_vals = np.linspace(lb[0], ub[0], tests_per_dim)
p2_vals = np.linspace(lb[1], ub[1], tests_per_dim)
all_param_vals = [x for x in itertools.product(p1_vals, p2_vals)]
    
for i, param_vals in enumerate(all_param_vals):
    param_sets[i, 0] = param_vals[0]  # Set beta encoding
    param_sets[i, 3] = param_vals[1]  # Set gamma CF

### Run/load grid search

In [ ]:
done = True

results = None
if done:
    with open('/scratch/jpazdera/cmr2/param_saves/sim2a/results.pkl', 'rb') as f:
        results = pkl.load(f)
else:
    try:
        with cluster_view(scheduler='sge', queue='RAM.q', num_jobs=101, cores_per_job=1) as view:
            results = view.map(eval_model, param_sets)
    except Exception:
        pass

    with open('/scratch/jpazdera/cmr2/param_saves/sim2a/results.pkl', 'wb') as f:
        pkl.dump(results, f, 2)

### Identify best-fitting parameters

In [ ]:
labels = ['SPC Composite', 'All', 'SPC_FR1', 'SPC_FRL4', 'PFR', 'PLIs', 'PLI Recency']

# Extract scores from results dictionaries
scores_spcv = np.array([r['err_spc_composite_v'] for r in results])
scores1 = np.array([r['err_v'] for r in results])
scores2 = np.array([r['err_spc_fr1_v'] for r in results])
scores3 = np.array([r['err_spc_frl4_v'] for r in results])
scores4 = np.array([r['err_pfr_v'] for r in results])
scores5 = np.array([r['err_pli_v'] for r in results])
scores6 = np.array([r['err_plir_v'] for r in results])
vscores = [scores_spcv, scores1, scores2, scores3, scores4, scores5, scores6]

scores_spca = np.array([r['err_spc_composite_a'] for r in results])
scores7 = np.array([r['err_a'] for r in results])
scores8 = np.array([r['err_spc_fr1_a'] for r in results])
scores9 = np.array([r['err_spc_frl4_a'] for r in results])
scores10 = np.array([r['err_pfr_a'] for r in results])
scores11 = np.array([r['err_pli_a'] for r in results])
scores12 = np.array([r['err_plir_a'] for r in results])
ascores = [scores_spca, scores7, scores8, scores9, scores10, scores11, scores12]

# Print best fitting parameter value, best score, and the index of the best fit
for i, s in enumerate(vscores):
    print(labels[i], all_param_vals[s.argmin()], s.min(), s.argmin())
print()
for i, s in enumerate(ascores):
    print(labels[i], all_param_vals[s.argmin()], s.min(), s.argmin())

### Plot error heatmap (2D)

In [ ]:
j = 0  # 0 = SPC Composite, 1 = Combined score, 2 = SPC_FR1 score, 3 = SPC_FRL4 score, 4 = PFR score, 5 = PLI score, 6 = PLI recency score

plt.subplot(1, 2, 1)
plt.imshow(np.sqrt(vscores[j]).reshape((tests_per_dim, tests_per_dim)), extent=(lb[1], ub[1], lb[0], ub[0]), cmap='RdBu', origin='lower', aspect='auto', interpolation='none', vmax=20, vmin=2.567)#, vmax=.16, vmin=.015)
plt.colorbar()
v_best = vscores[j].argmin()
plt.axhline(params[0], c='w', ls=':', lw=1.75)
plt.axvline(params[3], c='w', ls=':', lw=1.75)
plt.scatter(results[v_best]['params'][3], results[v_best]['params'][0], c='w', marker='*', s=75)
plt.xlim(lb[1], ub[1])
plt.ylim(lb[0], ub[0])
plt.title('Error (Visual)')
plt.xlabel(r'$\gamma_{CF}$')
plt.ylabel(r'$\beta_{enc}$')
plt.yticks([.34, .39, .44, .49, .53876], [.34, .39, .44, .49, .54])
plt.xticks([.38347, .43, .48, .53, .58, .63, .68], [.38, .43, .48, .53, .58, .63, .68])

plt.subplot(1, 2, 2)
plt.imshow(np.sqrt(ascores[j]).reshape((tests_per_dim, tests_per_dim)), extent=(lb[1], ub[1], lb[0], ub[0]), cmap='RdBu', origin='lower', aspect='auto', interpolation='none', vmax=20, vmin=2.567)#, vmax=.16, vmin=.015)
plt.colorbar()
a_best = ascores[j].argmin()
plt.axhline(params[0], c='w', ls=':', lw=2)
plt.axvline(params[3], c='w', ls=':', lw=2)
plt.scatter(results[a_best]['params'][3], results[a_best]['params'][0], c='w', marker='*', s=75)
plt.xlim(lb[1], ub[1])
plt.ylim(lb[0], ub[0])
plt.title('Error (Auditory)')
plt.xlabel(r'$\gamma_{CF}$')
plt.ylabel(r'$\beta_{enc}$')
plt.yticks([.34, .39, .44, .49, .53876], [.34, .39, .44, .49, .54])
plt.xticks([.38347, .43, .48, .53, .58, .63, .68], [.38, .43, .48, .53, .58, .63, .68])

plt.gcf().set_size_inches(13, 5)
plt.tight_layout()
plt.savefig('/home1/jpazdera/jupyter/ltpFR3/notebooks-modeling/sim2a_err.pdf')

### Maps of Intrusion Behavior

In [ ]:
plis = np.array([r['plis'] for r in results]).reshape((tests_per_dim, tests_per_dim))

# Visualize PLI distribution
#plt.figure()
#plt.histogram(plis)

# Take the log of PLIs to smooth the distribution
plis[plis <= 0] = plis[plis > 0].min()
plis = np.log(plis)

# Show new distribution
#plt.figure()
#plt.histogram(plis)

plt.subplot(121)
plt.imshow(plis, extent=(lb[1], ub[1], lb[0], ub[0]), cmap='RdBu_r', origin='lower', aspect='auto', interpolation='none')
plt.colorbar()
plt.axhline(params[0], c='w', ls=':', lw=1.75)
plt.axvline(params[3], c='w', ls=':', lw=1.75)
plt.xlim(lb[1], ub[1])
plt.ylim(lb[0], ub[0])
plt.title('log(PLIs)')
plt.xlabel(r'$\gamma_{CF}$')
plt.ylabel(r'$\beta_{enc}$')
plt.yticks([.34, .39, .44, .49, .53876], [.34, .39, .44, .49, .54])
plt.xticks([.38347, .43, .48, .53, .58, .63, .68], [.38, .43, .48, .53, .58, .63, .68])

plt.subplot(122)
plir = np.array([r['pli_recency'][0] for r in results]).reshape((tests_per_dim, tests_per_dim))
plt.imshow(plir, extent=(lb[1], ub[1], lb[0], ub[0]), cmap='RdBu_r', origin='lower', aspect='auto', interpolation='none')
plt.colorbar()
plt.axhline(params[0], c='w', ls=':', lw=1.75)
plt.axvline(params[3], c='w', ls=':', lw=1.75)
plt.xlim(lb[1], ub[1])
plt.ylim(lb[0], ub[0])
plt.title('PLI Recency')
plt.xlabel(r'$\gamma_{CF}$')
plt.ylabel(r'$\beta_{enc}$')
plt.yticks([.34, .39, .44, .49, .53876], [.34, .39, .44, .49, .54])
plt.xticks([.38347, .43, .48, .53, .58, .63, .68], [.38, .43, .48, .53, .58, .63, .68])

plt.gcf().set_size_inches(13, 5)
plt.tight_layout()
plt.savefig('/home1/jpazdera/jupyter/ltpFR3/notebooks-modeling/sim2a_intrusion_maps.pdf')

### Plot error heatmap (3D)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

j = 0  # 0 = SPC Composite, 1 = Combined score, 2 = SPC_FR1 score, 3 = SPC_FRL4 score, 4 = PFR score, 5 = PLI score, 6 = PLI recency score

fig = plt.figure()
ax = fig.add_subplot(121, projection='3d')
ax.plot_surface(X=np.array([p[0] for p in all_param_vals]).reshape((tests_per_dim, tests_per_dim)), Y=np.array([p[1] for p in all_param_vals]).reshape((tests_per_dim, tests_per_dim)), Z=np.sqrt(vscores[j]).reshape((tests_per_dim, tests_per_dim)), cmap='RdBu')
ax = fig.add_subplot(122, projection='3d')
ax.plot_surface(X=np.array([p[0] for p in all_param_vals]).reshape((tests_per_dim, tests_per_dim)), Y=np.array([p[1] for p in all_param_vals]).reshape((tests_per_dim, tests_per_dim)), Z=np.sqrt(ascores[j]).reshape((tests_per_dim, tests_per_dim)), cmap='RdBu')
plt.gcf().set_size_inches(10, 5)
plt.tight_layout()

# Advanced Plotting

In [ ]:
with open('../CMR2/target_stats_sim2a_v.json', 'r') as f:
    dv = json.load(f)
    
with open('../CMR2/target_stats_sim2a_a.json', 'r') as f:
    da = json.load(f)

with open('/scratch/jpazdera/cmr2/param_saves/sim2a/results.pkl', 'rb') as f:
    m = pkl.load(f)
    mv = m[np.argmin([x['err_spc_composite_v'] for x in m])]
    ma = m[np.argmin([x['err_spc_composite_a'] for x in m])]

In [ ]:
plot_spcs(dv, da, mv, ma)
plt.gcf().savefig('/home1/jpazdera/jupyter/ltpFR3/notebooks-modeling/sim2a_spcs.pdf')

In [ ]:
plot_nonspc(dv, da, mv, ma)
plt.gcf().savefig('/home1/jpazdera/jupyter/ltpFR3/notebooks-modeling/sim2a_nonspc.pdf')